# Monitoring and Muxing

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [ ]:
%matplotlib notebook
%run startup.py

from ophyd.sim import rand, det

## Data Acquisition

### Monitor `sig` asynchronously while counting `det` five times.

In [ ]:
def monitor_count(detectors, monitors, num=5, delay=1):
    "Monitor `monitors` while counting `detectors`."
    # Instantiate a normal 'count' plan.
    plan = bp.count(detectors, num=num, delay=delay)

    # Use a 'preprocessor', monitoring_during_wrapper, to add monitors.
    wrapped_plan = bpp.monitor_during_wrapper(plan, monitors)
    return (yield from wrapped_plan)

The ``TimePrinter`` callback prints a line whenever we capture a reading from `det` or `sig`.

In [ ]:
RE(monitor_count([det], [rand], delay=1), print)

### Access the saved data

The data is organized into two "streams", which we can access as tables where each row is an "event" -- a group of readings taken at the same time.

In [ ]:
header = db[-1]
header.table()  # shows the 'primary' stream by default

In [ ]:
header.table(stream_name='primary')  # equivalent to the above

In [ ]:
header.table(stream_name='flipper_monitor')

### Count two detectors synchronously while again monitoring `sig` asynchronously.

In [ ]:
RE(monitor_count([det, det2], [sig], delay=1), TimePrinter())

In [ ]:
header = db[-1]

The readings from `det` and `det2` are shown in the same table because their readings are synchronous.

In [ ]:
header.table()

### Plot data stream together

In [ ]:
fig, ax = plt.subplots()

ax.plot('time', 'det', data=header.table(), marker='o', label='det')
ax.plot('time', 'det2', data=header.table(), marker='o', label='det2')
ax.plot('time', 'flipper', data=header.table(stream_name='flipper_monitor'), marker='x', label='flipper')
ax.legend()

### Interpolate and normalize

In [ ]:
# Fetch all streams at once.
header.table(stream_name=db.ALL)

In [ ]:
# Make 'time' the index and sort on it.
header.table(stream_name=db.ALL).set_index('time').sort_index()

In [ ]:
# Interpolate using 'forward filling'.
header.table(stream_name=db.ALL).set_index('time').sort_index().ffill()

In [ ]:
data = header.table(stream_name=db.ALL).set_index('time').sort_index().ffill()
data['det'] * data['flipper']

In [ ]:
plt.figure()
(data['det'] * data['flipper']).plot()

## Exercises

1. Execute the ``monitor_count`` with a different ``delay`` parameter to verify that the readings from ``sig`` come at 5 Hz.